In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline




In [2]:
av = pd.read_csv('avocado.csv', index_col=0)

In [3]:
av.head()
# we need to find average price per day per region, since we have the same day for several cities in the same state
# how do we do that???? does it affect stationarity, how do we deal with it?

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
av.rename(columns={'4046':'small', '4225':'medium', '4770':'large'}, inplace=True)

In [5]:
av.drop(columns=['Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags'], inplace=True)

In [6]:
av.shape

(18249, 9)

# creating date time object in date column

In [7]:
av['Date'] = pd.to_datetime(av['Date'], format='%Y-%m-%d')

In [8]:
av.head()

,Date,AveragePrice,Total Volume,small,medium,large,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,conventional,2015,Albany


In [9]:
av.set_index('Date')

,AveragePrice,Total Volume,small,medium,large,type,year,region
Date,,,,,,,,
2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,conventional,2015,Albany
2015-12-20,1.35,54876.98,674.28,44638.81,58.33,conventional,2015,Albany
2015-12-13,0.93,118220.22,794.70,109149.67,130.50,conventional,2015,Albany
2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,conventional,2015,Albany
2015-11-29,1.28,51039.60,941.48,43838.39,75.78,conventional,2015,Albany
...,...,...,...,...,...,...,...,...
2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,organic,2018,WestTexNewMexico
2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,organic,2018,WestTexNewMexico
2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,organic,2018,WestTexNewMexico


# simple EDA

In [ ]:
sns.pairplot(av)

In [ ]:
# Function to plot each feature vs target variable
def plot_features(df, features):
    pp_rows = [features[i:i+4] for i in range(0, len(features), 4)]
    for row in pp_rows:
        pp = sns.pairplot(data=df, x_vars=row, y_vars='AveragePrice', kind='reg', height=3)
    return plt.show()

features = av.drop(['type', 'year', 'region', 'Date'], axis=1).keys()
plot_features(av, features)

In [ ]:
av.isnull().sum()

In [ ]:
av.groupby('region')['AveragePrice'].plot()

# removing regions and combining multiple cities into a single state

In [ ]:
av['region'].unique()

In [10]:
 stat = {'Albany':'ny', 'Atlanta':'ga', 'BaltimoreWashington':'md', 'Boise':'id','Boston':'ma',
         'BuffaloRochester':'ny', 'California':'ca', 'Charlotte':'nc', 'Chicago':'il', 'Detroit':'mi',
         'GrandRapids':'mi', 'GreatLakes':'greatlakes', 'HarrisburgScranton':'pa', 'HartfordSpringfield': 'ct',
         'Houston':'tx', 'Indianapolis':'in', 'Jacksonville':'fl', 'LasVegas':'nv', 'LosAngeles':'ca',
         'Louisville':'ky', 'MiamiFtLauderdale':'fl', 'Midsouth':'midsouth', 'Nashville':'tn',
         'NewOrleansMobile':'la', 'NewYork':'ny', 'Northeast':'northeast',
         'NorthernNewEngland':'northernnewengland', 'Orlando':'fl', 'Philadelphia':'pa', 'PhoenixTucson':'az',
         'Pittsburgh':'pa', 'Plains':'plains', 'Portland':'or', 'RaleighGreensboro':'nc', 
         'RichmondNorfolk':'va', 'Roanoke':'va', 'Sacramento':'ca', 'SanDiego':'ca', 'SanFrancisco':'ca',
         'Seattle':'wa', 'SouthCarolina':'sc', 'SouthCentral':'southcentral', 'Southeast':'southeast',
         'Spokane':'wa', 'StLouis':'mo', 'Syracuse':'ny', 'Tampa':'fl', 'TotalUS':'USA', 'West':'west', 
         'WestTexNewMexico':'nm', 'CincinnatiDayton':'oh', 'Columbus':'oh', 'DallasFtWorth':'tx', 'Denver':'co'}

# northeast, midsouth, southeast, south central, west, great lakes, northern new england, totalUS


'''aggregate the west ourselves and see if the pre-made aggregated west region correctly represents the data'''

'aggregate the west ourselves and see if the pre-made aggregated west region correctly represents the data'

In [11]:
cities = list(set(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico']) - set(['Northeast', 'Midsouth', 'Southeast', 'SouthCentral', 'West', 'GreatLakes', 'TotalUS', 'NorthernNewEngland', 'Plains']))


In [12]:
cities

['California',
 'Boston',
 'Charlotte',
 'CincinnatiDayton',
 'MiamiFtLauderdale',
 'Boise',
 'StLouis',
 'RaleighGreensboro',
 'PhoenixTucson',
 'BuffaloRochester',
 'Chicago',
 'NewYork',
 'LasVegas',
 'Philadelphia',
 'Seattle',
 'Louisville',
 'Portland',
 'Columbus',
 'GrandRapids',
 'Orlando',
 'DallasFtWorth',
 'Sacramento',
 'Albany',
 'Syracuse',
 'Detroit',
 'SanFrancisco',
 'Roanoke',
 'Houston',
 'SanDiego',
 'LosAngeles',
 'Jacksonville',
 'Atlanta',
 'Spokane',
 'Indianapolis',
 'Denver',
 'Nashville',
 'HartfordSpringfield',
 'Pittsburgh',
 'RichmondNorfolk',
 'SouthCarolina',
 'WestTexNewMexico',
 'Tampa',
 'BaltimoreWashington',
 'HarrisburgScranton',
 'NewOrleansMobile']

In [13]:
avocado = av[av['region'].isin(cities)].copy()

In [14]:
avocado.shape

(15207, 9)

In [15]:
avocado.replace({'region':stat}, inplace=True)

In [16]:
avocado.groupby('region')['AveragePrice'].describe()

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
az,338.0,1.224438,0.548283,0.46,0.6800,1.225,1.7200,2.62
ca,1690.0,1.487053,0.495607,0.53,1.1000,1.400,1.8075,3.25
co,338.0,1.218580,0.317915,0.60,0.9900,1.165,1.4300,2.16
ct,338.0,1.818639,0.468989,0.86,1.3900,1.800,2.2700,2.68
fl,1352.0,1.463624,0.376066,0.54,1.2075,1.440,1.7300,3.17
ga,338.0,1.337959,0.398297,0.62,1.0400,1.230,1.6200,2.75
id,338.0,1.348136,0.457911,0.58,1.0300,1.220,1.6375,2.79
il,338.0,1.556775,0.335439,0.70,1.2400,1.610,1.8000,2.30
in,338.0,1.313994,0.282894,0.77,1.0800,1.290,1.5275,2.10


In [17]:
avocado.groupby('region')['Total Volume'].describe()

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
az,338.0,5.788264e+05,5.974847e+05,4881.79,11449.3575,339075.580,1.138581e+06,2200550.27
ca,1690.0,1.087375e+06,1.867886e+06,3562.52,22347.4175,251731.110,7.659474e+05,11213596.29
co,338.0,4.109542e+05,3.940824e+05,6554.93,27517.2050,329377.845,7.663101e+05,1381528.74
ct,338.0,1.499128e+05,1.474348e+05,1494.63,8060.3800,111484.915,2.811244e+05,522565.33
fl,1352.0,1.857459e+05,2.258621e+05,84.56,3721.9050,53105.295,3.355664e+05,1310671.51
ga,338.0,2.621453e+05,2.631071e+05,3047.38,10989.8975,182673.440,4.907090e+05,957792.07
id,338.0,4.264257e+04,4.247940e+04,562.64,2212.7675,28988.585,8.132399e+04,136377.55
il,338.0,3.955690e+05,3.846002e+05,9910.84,29956.2450,231780.020,7.639302e+05,1802646.35
in,338.0,8.953666e+04,8.911021e+04,964.25,3746.8175,60553.145,1.680288e+05,335442.41


# creating df for different types of avocados

In [18]:
conventional = avocado[avocado['type'] == 'conventional']

In [19]:
organic = avocado[avocado['type'] == 'organic']

In [20]:
organic.head()

,Date,AveragePrice,Total Volume,small,medium,large,type,year,region
0,2015-12-27,1.83,989.55,8.16,88.59,0.0,organic,2015,ny
1,2015-12-20,1.89,1163.03,30.24,172.14,0.0,organic,2015,ny
2,2015-12-13,1.85,995.96,10.44,178.70,0.0,organic,2015,ny
3,2015-12-06,1.84,1158.42,90.29,104.18,0.0,organic,2015,ny
4,2015-11-29,1.94,831.69,0.00,94.73,0.0,organic,2015,ny


In [21]:
conventional.head()

,Date,AveragePrice,Total Volume,small,medium,large,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,conventional,2015,ny
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,conventional,2015,ny
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,conventional,2015,ny
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,conventional,2015,ny
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,conventional,2015,ny


In [ ]:
organic.groupby('region')['AveragePrice'].plot()

In [ ]:
conventional.groupby('region')['AveragePrice'].plot()

# dickey fuller test

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
grrt = np.log(av['AveragePrice'])

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

In [ ]:
avocado['AveragePrice'].diff(periods=1)

In [ ]:
difference(av['AveragePrice'])

In [ ]:
test = adfuller((np.log(avocado['AveragePrice']).diff(periods=1))[1:])
# test = adfuller(diff['Differenced Observations'][1:-1])
dfoutput = pd.Series(test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
print(dfoutput)